In [23]:
import pandas as pd
from core.helpers import yakeHelperFunctions as yhf
import glob
from bs4 import BeautifulSoup as bs
from tqdm import tqdm

In [24]:
job_dir = "/data/AllProjectIA/Scraping/projet/"

# get all text of jobs

In [25]:
html_contents_emploiscm = []
html_contents_jobinfos = []
for file_name in tqdm(sorted(glob.glob(f'{job_dir}emploi.cm/*.html'))):
    with open(file_name, 'r') as f:
        html_contents_emploiscm.append(f.read())
for file_name in tqdm(sorted(glob.glob(f'{job_dir}jobinfocamer.com/*.html'))):
    with open(file_name, 'r') as f:
        html_contents_jobinfos.append(f.read())

100%|██████████| 7980/7980 [00:00<00:00, 15692.05it/s]


In [26]:
print(f"nous avons charger {len(html_contents_emploiscm)} HTML files de emplois.cm")
print(f"nous avons charger {len(html_contents_jobinfos)} HTML files de job info")

nous avons charger 185 HTML files de emplois.cm
nous avons charger 7980 HTML files de job info


# remove HTML Tags

In [27]:
def remove_html_tags(html):
   soup = bs(html, "html.parser")
   for data in soup(['style', 'script']):
       data.decompose()
   return ' '.join(soup.stripped_strings)



In [28]:
def remove_html_tags_from_description(html):
   soup = bs(html, "html.parser")
   divs = soup.findAll('div', class_='inner clearfix')
   return divs

def remove_html_tags_from_description2(html):
   soup = bs(html, "html.parser")
   divs = soup.findAll('div', class_='text-justify margin-bottom-30')
   return divs

In [29]:
contents = []
for html in tqdm(html_contents_emploiscm):
    divs = remove_html_tags_from_description(html)
    for data in divs[2](['style', 'script']):
       data.decompose()
    text =' '.join(divs[2].stripped_strings)
    text  = text.replace(u'\xa0', u' ')
    contents.append(text)
for html in tqdm(html_contents_jobinfos):
    divs = remove_html_tags_from_description2(html)
    for data in divs[0](['style', 'script']):
       data.decompose()
    text =' '.join(divs[0].stripped_strings)
    text  = text.replace(u'\xa0', u' ')
    contents.append(text)


100%|██████████| 7980/7980 [02:23<00:00, 55.48it/s]


In [30]:
job_data = pd.DataFrame(contents, columns=['job'])

In [31]:
job_data.to_csv("dataset/job.csv", index=False)

# cut sample (1000 rows)

In [32]:
job_sample= job_data.job[-1000:].astype(str)



# extract set of keywords from each post

In [33]:
keywords = yhf.keywords_yake(job_sample)
keywords_tokenized = yhf.tokenizing_after_yake(keywords)
keywords_sets = [set(job) for job in keywords_tokenized]

In [34]:
keywords_tokenized[:5]

[['competences',
  'requises',
  'être',
  'vos',
  'missions',
  'prospection',
  'direction',
  'competences',
  'requises',
  'requises',
  'être',
  'titulaire',
  'vente',
  'les',
  'candidats',
  'titulaire',
  'd',
  'un',
  'bac',
  'être',
  'titulaire',
  'd',
  'un',
  'télécommunications',
  'excellente',
  'élocution',
  'les',
  'candidats',
  'ayant',
  'secteur',
  'des',
  'télécommunications',
  'des',
  'télécommunications',
  'excellente',
  'vente',
  'réalisation',
  'des',
  'vente',
  'remontée',
  'des',
  'ans',
  'd',
  'expérience',
  'professionnelle',
  'pourront',
  'faire',
  'parvenir',
  'bien',
  'vouloir',
  'préciser',
  'suivi',
  'des',
  'objectifs',
  'nouveaux',
  'clients',
  'fidélisation',
  'portefeuille',
  'clients',
  'présentation',
  'clients',
  'présentation',
  'des'],
 ['offre',
  'd',
  'emploi',
  'nous',
  'offre',
  'd',
  'emploi',
  'personnes',
  'talentueuses',
  'qui',
  'talentueuses',
  'qui',
  'aimeraient',
  'qui',
 

In [35]:
keywords[:5]

['COMPETENCES REQUISES Être VOS MISSIONS prospection direction COMPETENCES REQUISES REQUISES Être titulaire vente Les candidats titulaire d’un BAC Être titulaire d’un Télécommunications excellente élocution Les candidats ayant secteur des Télécommunications des Télécommunications excellente vente réalisation des vente remontée des ans d’expérience professionnelle pourront faire parvenir bien vouloir préciser suivi des objectifs nouveaux clients fidélisation portefeuille clients présentation clients présentation des ',
 "OFFRE D'EMPLOI Nous OFFRE D'EMPLOI personnes talentueuses qui talentueuses qui aimeraient qui aimeraient renforcer aimeraient renforcer notre renforcer notre équipe Web Description Analyses recherche des personnes des personnes talentueuses D'EMPLOI Nous sommes Description Analyses personnes talentueuses talentueuses qui qui aimeraient aimeraient renforcer renforcer notre notre équipe équipe et construire construire le futur ",
 "OFFRE D'EMPLOI Action Contre la Faim D'E

In [36]:
job_data[-1000:]

,job
7165,CHARGE(E) D’AFFAIRES - JobInfoCamer.com Email:...
7166,Développeur Front End expérimenté - JobInfoCam...
7167,SUPERVISEUR MSA - JobInfoCamer.com Email: aide...
7168,Administrateur des Ventes junior - JobInfoCame...
7169,Teachers - JobInfoCamer.com Email: aide@jobinf...
...,...
8160,Rédacteurs web - JobInfoCamer.com Email: aide@...
8161,Cadre Comptable - JobInfoCamer.com Email: aide...
8162,Comptable - JobInfoCamer.com Email: aide@jobin...
8163,CHEF DE CHANTIER - JobInfoCamer.com Email: aid...
